In [1]:
import argparse
import qlib
import ruamel.yaml as yaml
from qlib.utils import init_instance_by_config
import pandas as pd
import torch
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)

In [2]:
parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--seed", type=int, default=1000, help="random seed")
parser.add_argument("--config_file", type=str, default="backtest.yaml", help="config file")
parser.add_argument("-f")
args = parser.parse_args()


seed = args.seed
config_file = args.config_file

# set random seed
with open(config_file) as f:
    config = yaml.safe_load(f)

# seed_suffix = "/seed1000" if "init" in config_file else f"/seed{seed}"
seed_suffix = ""
config["task"]["model"]["kwargs"].update(
    {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
)

# initialize workflow
qlib.init(
    provider_uri=config["qlib_init"]["provider_uri"],
    region=config["qlib_init"]["region"],
)

[3453:MainThread](2021-10-30 00:05:03,354) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[3453:MainThread](2021-10-30 00:05:03,356) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[3453:MainThread](2021-10-30 00:05:03,545) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[3453:MainThread](2021-10-30 00:05:03,546) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data')}


In [3]:
latest_dataset = init_instance_by_config(config["task"]["dataset"])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[3453:MainThread](2021-10-30 00:08:20,267) INFO - qlib.timer - [log.py:113] - Time cost: 196.705s | Loading data Done
[3453:MainThread](2021-10-30 00:08:22,577) INFO - qlib.timer - [log.py:113] - Time cost: 1.943s | FilterCol Done
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.8/dist

In [4]:
model = init_instance_by_config(config["task"]["model"])

Please install necessary libs for CatBoostModel.
Please install necessary libs for XGBModel, such as xgboost.


[3453:MainThread](2021-10-30 00:08:39,579) WARNING - qlib.TRA - [pytorch_tra.py:100] - `eval_train` will be ignored when using TRA.router
[3453:MainThread](2021-10-30 00:08:39,581) WARNING - qlib.TRA - [pytorch_tra.py:132] - logdir output/us_158_nassp_epoch100 is not empty
[3453:MainThread](2021-10-30 00:08:39,583) INFO - qlib.TRA - [pytorch_tra.py:141] - init TRAModel...
[3453:MainThread](2021-10-30 00:08:39,587) INFO - qlib.TRA - [pytorch_tra.py:171] - # model params: 59520
[3453:MainThread](2021-10-30 00:08:39,588) INFO - qlib.TRA - [pytorch_tra.py:172] - # tra params: 5606


RNN(
  (rnn): LSTM(20, 64, num_layers=2, batch_first=True)
  (W): Linear(in_features=64, out_features=64, bias=True)
  (u): Linear(in_features=64, out_features=1, bias=False)
)
TRA(
  (predictors): Linear(in_features=128, out_features=3, bias=True)
  (router): LSTM(3, 32, batch_first=True)
  (fc): Linear(in_features=160, out_features=3, bias=True)
)


In [5]:
state_dict = torch.load("/home/qlib_test/examples/benchmarks/TRA/output/us_158_nassp_epoch100/model.bin", map_location="cpu")["model"]

In [6]:
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [7]:
model.fitted = True

In [12]:
pred = model.predict(latest_dataset)
reset_df = pred.reset_index('instrument')

100%|████████████████████████████████████████████████████| 1405/1405 [03:25<00:00,  6.84it/s]
[3453:MainThread](2021-10-30 00:23:30,929) INFO - qlib.TRA - [pytorch_tra.py:516] - test metrics: {'MSE': 0.16716719063699798, 'MAE': 0.3333444516749473, 'IC': -0.002645107436722937, 'ICIR': -0.023146544596854512}


In [13]:
import qlib
from qlib.data import D
qlib.init(provider_uri="/home/us_data", region="us")

d_instruments = D.instruments("nassp")

[3453:MainThread](2021-10-30 00:23:30,979) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[3453:MainThread](2021-10-30 00:23:30,981) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[3453:MainThread](2021-10-30 00:23:30,983) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[3453:MainThread](2021-10-30 00:23:30,985) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data')}


In [18]:
df = D.features(d_instruments, ["$open","$factor"], freq="day")

In [197]:
import time
import numpy as np

def backtest(top_k, max_buying, fee, p=False, early_stop=True, money=1000.0):
    st = time.time()
    indexs = reset_df.index.drop_duplicates()
    cnt = 0 
    # money = 100
    current_buying = {}
    num_buy = 0
    loss_cut = 0.95
    my_stack = None

    for idx in indexs:
        cnt+=1
        today_buy_list = []
        today_sell_list = []
        today_earn = 0
        
        try:
            if p:
                print("---------------------------------")
                print("day ", cnt, idx)
                print("---------------------------------")
                
            ## predict today score
            if type(reset_df.loc[idx]['score']) is np.float32:
                stack = reset_df.loc[idx]
            else:
                stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)
                # stack = stack[stack['score'] < -0.17]
            
            ## update current stocks' prices, money
            # print("money, can_buy_money:", money, can_buy_money)
            can_buy_money = money
            current_buying_list = list(current_buying.keys())
            for cur_stock in current_buying_list:
                today_stock_price = df.loc[cur_stock,idx][0] / df.loc[cur_stock,idx][1]
                today_stock_price = today_stock_price.round(2)
                current_buying[cur_stock][2] = today_stock_price.round(2)
                current_buying[cur_stock][4] = (current_buying[cur_stock][3] * today_stock_price).round(2)
                ### update current total money
                if not np.isnan(current_buying[cur_stock][4]):
                    can_buy_money += current_buying[cur_stock][4].round(2)
            # print("money, can_buy_money:", money, can_buy_money)
            
            if p:
                print("current stocks: ", current_buying)
                
            ## sorting current stocks' score
            my_stack = stack[stack['instrument'].isin(current_buying_list)]
            my_stack = my_stack.sort_values(by='score' , ascending=True)
            my_stack_list = list(my_stack['instrument'])
            
            # print("current stocks' score:", my_stack_list)
            
            ## select buy_list and sell_list
            today_buy_cnt = 0
            for x in stack['instrument']:
                if today_buy_cnt >= top_k:
                    break
                cur_stock = x
                
                if cur_stock in my_stack_list:
                    # today_buy_list.append(cur_stock)
                    # today_sell_list.append(cur_stock)
                    today_buy_cnt+=1
                    if not np.isnan(current_buying[cur_stock][4]):
                        can_buy_money -= current_buying[cur_stock][4].round(2)
                    continue
                
                ### assert we can buy this stock
                today_stock_price = df.loc[cur_stock,idx][0] / df.loc[cur_stock,idx][1]
                today_stock_price = today_stock_price.round(2)
                if today_stock_price > can_buy_money / (max_buying - today_buy_cnt):
                    continue
                ### assert price is not nan
                if np.isnan(today_stock_price):
                    continue
                
                ### sell
                if num_buy >= max_buying:
                    last_stock = my_stack_list[-1]
                    num_last_stock = current_buying[last_stock][0]

                    num_buy -= 1
                    today_sell_list.append(last_stock)
                    if num_last_stock == 1:
                        earn = current_buying[last_stock][4]
                        del current_buying[last_stock]
                        my_stack_list = my_stack_list[:-1]
                    else:
                        current_buying[last_stock][0] -= 1
                        current_buying[last_stock][3] -= sell_total
                        current_buying[last_stock][4] -= sell_total*end
                        earn = sell_total*end
                    if not np.isnan(earn):
                        money += earn.round(2)*(1.0-fee)

                ### buy
                total = 1
                while(1):
                    total_price = (total+1)*today_stock_price
                    if total_price < can_buy_money / (max_buying - today_buy_cnt):
                        total+=1
                    else:
                        break
                total_price = total*today_stock_price
                total_price = total_price.round(2)
                
                num_buy+=1
                today_buy_cnt += 1
                today_buy_list.append(cur_stock)
                if cur_stock not in current_buying:
                    current_buying[x] = [1, today_stock_price, today_stock_price, total, total_price] #num, start_price, current_price, total, total_price
                else:
                    current_buying[x][0] += 1
                    current_buying[x][3] += total
                    current_buying[x][4] += total_price
                if not np.isnan(total_price):
                    money -= total_price.round(2)*(1.0+fee)
                    can_buy_money -= total_price.round(2)*(1.0+fee)
            if p:
                print("buy: ", today_buy_list)
                print("sell: ", today_sell_list)
            # print("current stocks: ", current_buying)
            
            ## caluate current_money
            current_money = money
            for c in current_buying:
                total_price = current_buying[c][4]
                current_money += total_price
            # if money<0:
                # print("?")
            if p:
                print("current stocks: ", len(current_buying), current_buying)
                print("current usable money: ", money)
                print("current money: ", current_money)
        except Exception as e:
            print(e)
            if early_stop and cnt >= 800:
                print("---------------------------------")
                print("config: ", top_k, max_buying)
                print("day ", cnt, idx)
                print("current stocks: ", len(current_buying), current_buying)
                print("current money: ", current_money)
                print("time:", time.time() - st)
                break
            continue
        if early_stop and cnt >= 800:
            print("---------------------------------")
            print("config: ", top_k, max_buying)
            print("day ", cnt, idx)
            print("current stocks: ", len(current_buying), current_buying)
            print("current usable money: ", money)
            print("current money: ", current_money)
            print("time:", time.time() - st)
            break
    return current_money

In [198]:
# for a in range(1,7):
    # backtest(a, a, 0.0007, p=False, money=1000.0)
backtest(1, 1, 0.0007, p=False, money=1000.0)
# backtest(2, 2, 0.0007, p=False, money=1000.0)

---------------------------------
config:  1 1
day  800 2018-03-07 00:00:00
current stocks:  1 {'CSRA': [1, 40.65, 40.61, 182, 7391.02]}
current usable money:  -0.5671969998966233
current money:  7390.452803000104
time: 4.292951822280884


7390.452803000104

In [65]:
backtest(5, 5, 0.0007, p=False, money=3000)

---------------------------------
config:  5 5
day  800 2018-03-07 00:00:00
5 {'CSRA': [2, 40.55, 40.609997], 'PXD': [1, 168.4, 168.4], 'NOC': [1, 337.98996, 337.98996], 'HAS': [1, 95.45, 95.45]}
current money:  21908.450890992786
time: 5.150775194168091


21908.450890992786

In [66]:
backtest(5, 5, 0.0007, p=False, money=2000)

---------------------------------
config:  5 5
day  800 2018-03-07 00:00:00
5 {'CSRA': [2, 40.55, 40.609997], 'PXD': [1, 168.4, 168.4], 'NOC': [1, 337.98996, 337.98996], 'HAS': [1, 95.45, 95.45]}
current money:  14354.32257563268
time: 5.206586599349976


14354.32257563268